In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!python -m pip install git+https://github.com/huggingface/transformers
! pip install openpyxl
!pip install qwen-vl-utils

  Cloning https://github.com/huggingface/transformers to c:\users\atharva sankhe\appdata\local\temp\pip-req-build-2norm9al
  Resolved https://github.com/huggingface/transformers to commit c7a109ec81fb38fdd77d1d0543a9233bc6bf4d09
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------------------- 41.5/41.5 kB 2.0 MB/s eta 0:00:00
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 57.7/57.7 kB 3.0 MB/s eta 0:00:00

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers 'C:\Users\Atharva Sankhe\AppData\Local\Temp\pip-req-build-2norm9al'

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   --------- ------------------------------ 61.4/250.9 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 250.9/250.9 kB 3.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached pillow-11.0.0-cp311-cp311-win_amd64.whl.metadata (9.3 kB)
   ---------------------------------------- 0.0/25.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/25.8 MB 6.0 MB/s eta 0:00:05
    --------------------------------------- 0.6/25.8 MB 6.3 MB/s eta 0:00:05
   - -------------------------------------- 0.7/25.8 MB 5.0 MB/s eta 0:00:06
   - -------------------------------------- 0.8/25.8 MB 5.0 MB/s eta 0:00:06
   - -------------------------------------- 1.0/25.8 MB 4.5 MB/s eta 0:00:06
   - -------------------------------------- 1.2/25.8 MB 4.4 MB/s eta 0:00:06
   -- ------------------------------------- 1.3/25.8 MB 4.2 MB/s eta 0:00:06
   -- ------------------------------------- 1.5/25.8 MB 4.1 MB/s eta 0:00:07
   -- ------------------------------------- 1.6/25.8 MB 3.9 MB/s eta 0:00:07
   -- ------------------------------------- 1.8/25.8 MB 3.9 MB/s eta 0:00:07
   -- ------------------------------------- 1.9/25.8 MB 3.8 MB/s eta 0:00:07
   --- ---


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import re
import openpyxl
import cv2
from PIL import Image
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import numpy as np
import os

In [9]:
# Load the Qwen2 model and processor
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    torch_dtype="auto",
    # device_map="auto",
)

model.to("cpu")

processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

KeyboardInterrupt: 

In [ ]:
model.save_pretrained("")
processor.save_pretrained("")

In [5]:
messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image_url": "https://www.akospices.com/cdn/shop/articles/Indian_Grocery_Store_in_UK_Online.jpg?v=1584438200"  # Description for internal use
                },
                {
                    "type": "text",
                    "text": """This image contains fruits, vegetables, or packaged products.
                    Please analyze the image and provide:
                    - For products or fruits or vegetables:
                        - Product Name
                        - Product Category
                        - Product Quantity
                        - Product Count
                        - Expiry Date (if available)
                        - Freshness Index (based on visual cues)
                        - Estimated Shelf Life"""
                }
            ]
        }
    ]

In [11]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")


In [ ]:
output_ids = model.generate(**inputs, max_new_tokens=1024)

# Decode the generated output
generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(inputs.input_ids, output_ids)
]
output_text = processor.batch_decode(
    generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
)[0]

In [13]:
print(output_text)

The image contains a variety of packaged products, including:

1. **Fruits and Vegetables:**
   - **Product Name:** Mangoes
   - **Product Category:** Fruits
   - **Product Quantity:** 2
   - **Product Count:** 2
   - **Expiry Date:** Not visible
   - **Freshness Index:** High, as the mangoes appear fresh and vibrant.

2. **Packaged Products:**
   - **Product Name:** Maggi 2 Minute Noodles
   - **Product Category:** Noodles
   - **Product Quantity:** 1
   - **Product Count:** 1
   - **Expiry Date:** Not visible
   - **Freshness Index:** High, as the noodles appear fresh and well-packed.

3. **Packaged Products:**
   - **Product Name:** Aashirvaad Ready Meals
   - **Product Category:** Ready Meals
   - **Product Quantity:** 1
   - **Product Count:** 1
   - **Expiry Date:** Not visible
   - **Freshness Index:** High, as the ready meals appear fresh and well-packed.

4. **Packaged Products:**
   - **Product Name:** Taj Mahal Special Atta
   - **Product Category:** Flour
   - **Product Qua

In [10]:
product_pattern = re.compile(
        r"Product Name\s*:\s*(.*?)\s*"
        r"Product Category\s*:\s*(.*?)\s*"
        r"Product Quantity\s*:\s*(.*?)\s*"
        r"Product Count\s*:\s*(.*?)\s*"
        r"Expiry Date\s*:\s*(.*?)\s*"
        r"Freshness Index\s*:\s*(.*?)\s*"
        r"Shelf Life\s*:\s*(.*?)\s*",
        re.IGNORECASE | re.DOTALL
)

# Extract packaged product information using regex
product_match = re.search(product_pattern, output_text[0])

if product_match:
    product_name = product_match.group(1).strip()
    category = product_match.group(2).strip()
    quantity = product_match.group(3).strip()
    count = product_match.group(4).strip()
    expiry_date = product_match.group(5).strip()
    freshness_index = product_match.group(6).strip()
    shelf_life = product_match.group(7).strip()
else:
    # If no match is found, default to "N/A"
    product_name = category = quantity = count = expiry_date = freshness_index = shelf_life = "N/A"